# Comparing the robustness of U-Net, LinkNet and FPN towards label noise for refugee dwelling extraction from satellite imagery

For license reason, the data folder only includes 10% of data without georeference information.<br>
The script can be helpful for researchers of interest to develop algorithms for their own tasks. <br>
<br>
Author: Yunya Gao<br>
Affiliation:<br>
Christian Doppler Laboratory for geospatial and EO-based humanitarian technologies (GEOHUM)<br>
Department of Geoinformatics – Z_GIS<br>
Paris Lodron University of Salzburg<br>
Salzburg, Austria<br>
Last Update: 2022-04-29<br>




# Training models

## Google login and install packages

In [ ]:
# Define current working direction when in google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install packages
!pip install patchify
!pip install earthpy
!pip install segmentation_models
!pip install osgeo

     |████████████████████████████████| 1.4 MB 8.6 MB/s 
     |████████████████████████████████| 19.3 MB 405 kB/s 
     |████████████████████████████████| 1.0 MB 60.2 MB/s 
     |████████████████████████████████| 6.3 MB 33.8 MB/s 
     |████████████████████████████████| 16.7 MB 41.7 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
  ERROR: Failed building wheel for osgeo
  Running setup.py clean for osgeo
Failed to build osgeo
    Running setup.py install for osgeo ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-dot92fnz/osgeo_9d98d877d9974b3e94070749aae28a78/setup.py'"'"'; __file__='"'"'/tmp/pip-install-dot92fnz/osgeo_9d98d877d9974b3e94070749aae28a78/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');

## Paths, Libraries and Global variables

### Paths

In [ ]:
import os

# Define root path 
# e.g. /content/drive/MyDrive/Dwelling_Extraction/
path_base = "root path"

# Define folders
data_folder = "Data"
pred_folder = "Prediction"
weights_folder = "Weights"

# Get the current working directory
os.chdir(path_base)
print("Current working path:{}".format(path_base))

path_data_ = os.path.join(path_base, data_folder)
print("Output data path:{}".format(path_data_))

path_pred_ = os.path.join(path_base, pred_folder)
print("Prediction path:{}".format(path_pred_))

path_weights_ = os.path.join(path_base, weights_folder)
print("Weights path:{}".format(path_weights_))

Current working path:/content/drive/MyDrive/PhD_Research/Codes_ForTraining_SampleData
Output data path:/content/drive/MyDrive/PhD_Research/Codes_ForTraining_SampleData/Data
Prediction path:/content/drive/MyDrive/PhD_Research/Codes_ForTraining_SampleData/Prediction
Weights path:/content/drive/MyDrive/PhD_Research/Codes_ForTraining_SampleData/Weights


### Libraries

In [ ]:
# Basic functions
import numpy as np
import pandas as pd
from patchify import patchify, unpatchify

# Data processing functions
from Data_Preparation import *

# Import deep learning framework and models
import tensorflow as tf
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
print(f"Tensor Flow Version: {tf.__version__}")

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import MeanIoU

import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

from segmentation_models.metrics import f1_score, iou_score
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models import Unet, PSPNet, Linknet, FPN
from segmentation_models import get_preprocessing

from Model_And_SetUp import *

# Visualization
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline 
%pylab inline 

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

GPU is available
Tensor Flow Version: 2.8.0
Populating the interactive namespace from numpy and matplotlib


### Global variables

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Change this part for different tasks
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# Define important global variables
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
ignore_img_value = 65535
ignore_lab_value = 3
nbands_all = 4
figure_size = 50

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# Hyperparameters of model
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# The original patches can be further processed in this section based on different tasks.
patchsize = 128
nclass = 2
band_selection = ["R", "G", "B"] # can be other combinations, e.g. ["R", "G", "NIR"], ["R", "G", "B", "NIR"] 
nbands = len(band_selection)

epoch = 200
batchsize = 32

metrics_list = [MeanIoU(nclass)] # MeanIoU f1_score
activation_func = "softmax"
loss_func = balanced_cross_entropy

lr = 4e-4
OPT = Adam(learning_rate=lr, decay=lr/epoch)

# users can define callbacks based on their own needs
callbacks = []

# Available choices of backbones
backbone_vgg = ["vgg16", "vgg19"]
backbone_resnet = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']
backbone_seresnet = ['seresnet18', 'seresnet34', 'seresnet50', 'seresnet101', 'seresnet152']
backbone_resnext = ['resnext50', 'resnext101']
backbone_seresnext = ['seresnext50', 'seresnext101']
backbone_senet = ['senet154']
backbone_densenet = ['densenet121', 'densenet169', 'densenet201']
backbone_inception = ['inceptionv3', 'inceptionresnetv2']
backbone_mobilenet = ['mobilenet', 'mobilenetv2']
backbone_efficientnet = ['efficientnetb0', 'efficientnetb1', 'efficientnetb2', 'efficientnetb3', 'efficientnetb4', 'efficientnetb5', 'efficientnetb6', 'efficientnetb7']
backbone_all = [*backbone_vgg, *backbone_resnet, *backbone_seresnet, *backbone_resnext, *backbone_seresnext, *backbone_senet, *backbone_densenet, *backbone_inception, *backbone_mobilenet, *backbone_efficientnet]

## Running Models

In [ ]:
# Define dataset
# The folder of example data: ETH_KuleTiekidi_20170324 
# For license reason, this folder only includes 10% of data without georeference information 
EXP = "Original" # Original: directly use produced labels from past humanitarian operations
country = "ETH"
study_site = "KuleTierkidi" 
date = "20170324" 

# Define research area and date
data_name = country + "_" + study_site + "_" + date
path_data = os.path.join(path_data_, data_name)
print(path_data)

# Define path of weights
pretrained_weights = "imagenet"
weights_folder = "Noisy_RGB_ImageNet"

path_weights = os.path.join(path_weights_, data_name)
create_new_folder(path_weights, weights_folder)
print(path_weights)

# Load patches
tr_img, tr_lab, va_img, va_lab = get_training_validation_patch(path_data, EXP)
# Select image bands to be trained
X_tr, X_va, Y_tr, Y_va = select_image_bands(tr_img, tr_lab, va_img, va_lab, nbands, band_selection)

# Define backbones and initial weights
backbone_list = ["vgg16", "vgg19", "resnet18", "resnet34", "densenet121", "densenet169", "inceptionv3", "inceptionresnetv2", "mobilenet", "mobilenetv2", "efficientnetb0", "efficientnetb1"]

/content/drive/MyDrive/PhD_Research/Codes_ForTraining_SampleData/Data/ETH_KuleTierkidi_20170324
/content/drive/MyDrive/PhD_Research/Codes_ForTraining_SampleData/Weights/ETH_KuleTierkidi_20170324
Shape of training image patches: (829, 128, 128, 4)
Shape of training label patches: (829, 128, 128, 2)
Shape of validation image patches: (93, 128, 128, 4)
Shape of validation label patches: (93, 128, 128, 2)
Percent of target objects in training label: (0.07667076, 0.92332923)
Percent of target objects in validation label: (0.07453771, 0.9254623)
X_tr.shape: (829, 128, 128, 3), X_va.shape:(93, 128, 128, 3)


### 2.1 U-Net

In [ ]:
# Define models and initial weights
model_name = "unet"
backbone_list = ["vgg16", "vgg19", "resnet18", "resnet34", "densenet121", "densenet169", "inceptionresnetv2", "inceptionv3", "mobilenet", "mobilenetv2", "efficientnetb0", "efficientnetb1"]

for backbone in backbone_list:
    # run model
    model_type = model_name + "_" + backbone
    print("Starting training:", model_type)
    history, model = run_unet_0(backbone, pretrained_weights, X_tr, Y_tr, X_va, Y_va, callbacks, activation_func, loss_func, metrics_list, OPT, patchsize, nclass, nbands, epoch, batchsize)

    # save weights and history
    save_weights_history(path_weights, weights_folder, model_type, model, history)
    # save weights and history
    save_weights_history(path_weights, weights_folder, model_type, model, history)

    # plot training and validation accuracy and loss 
    plot_history_direct_from_model(history)

### 2.2 FPN

In [ ]:
model_name = "fpn"
backbone_list = ["vgg16", "vgg19", "resnet18", "resnet34", "densenet121", "densenet169", "inceptionresnetv2", "inceptionv3", "mobilenet", "mobilenetv2", "efficientnetb0", "efficientnetb1"]

for backbone in backbone_list:
    # run model
    model_type = model_name + "_" + backbone
    print("Starting training:", model_type)
    history, model = run_fpn(backbone, pretrained_weights, X_tr, Y_tr, X_va, Y_va, callbacks, activation_func, loss_func, metrics_list, OPT, patchsize, nclass, nbands, epoch, batchsize)

    # save weights and history
    save_weights_history(path_weights, weights_folder, model_type, model, history)
    # save weights and history
    save_weights_history(path_weights, weights_folder, model_type, model, history)

    # plot training and validation accuracy and loss 
    plot_history_direct_from_model(history)

### 2.3 Linknet

In [ ]:
model_name = "linknet"
backbone_list = ["vgg16", "vgg19", "resnet18", "resnet34", "densenet121", "densenet169", "inceptionresnetv2", "inceptionv3", "mobilenet", "mobilenetv2", "efficientnetb0", "efficientnetb1"]

for backbone in backbone_list:
    # run model
    model_type = model_name + "_" + backbone
    print("Starting training:", model_type)
    history, model = run_linknet(backbone, pretrained_weights, X_tr, Y_tr, X_va, Y_va, callbacks, activation_func, loss_func, metrics_list, OPT, patchsize, nclass, nbands, epoch, batchsize)

    # save weights and history
    save_weights_history(path_weights, weights_folder, model_type, model, history)
    # save weights and history
    save_weights_history(path_weights, weights_folder, model_type, model, history)

    # plot training and validation accuracy and loss 
    plot_history_direct_from_model(history)